# Biopython covid notebook tutorial

Este tutorial básico le muestra como identificar una "secuencia desconocida" de ADN/ARN que deriva de un genoma de coronavirus. Este tutorial usa [Biopython](https://github.com/biopython/biopython) (llamando a algunas herramientas) para identificar y comenzar a caracterizar esta secuencia.

## Config

Importar y verificar versiones

In [ ]:
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install biopython
except ImportError:
    pass

     |████████████████████████████████| 2.3 MB 7.7 MB/s 


In [ ]:
import os
import sys

from urllib.request import urlretrieve

import Bio
from Bio import SeqIO, SearchIO, Entrez
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.Blast import NCBIWWW
from Bio.Data import CodonTable

print("Python version:", sys.version_info)
print("Biopython version:", Bio.__version__)

Python version: sys.version_info(major=3, minor=7, micro=13, releaselevel='final', serial=0)
Biopython version: 1.79


Input file

In [ ]:
input_file = "unknown-sequence.fa"

fasta_loc = ("https://raw.githubusercontent.com/chris-rands/"
             "biopython-coronavirus/master/unknown-sequence.fa")

if not os.path.exists(input_file):
    urlretrieve(fasta_loc, input_file)

In [ ]:
!cat unknown-sequence.fa | wc -l

429


## Propiedades básicas




In [ ]:
for record in SeqIO.parse(input_file, "fasta"):
    print(record.id)

Unknown_sequence


Solo hay una secuencia con el encabezado "Unknown_sequence". No estamos tratando con muchos cromosomas, scaffolds o contigs.

Extraer la secuencia

In [ ]:
record = SeqIO.read(input_file, "fasta")

In [ ]:
record.seq

Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA')

In [ ]:
print("Sequence length (bp)", len(record))

Sequence length (bp) 29903


La longitud de la secuencia es ~30Kb, si esta secuencia representa un organismo individual, entonces es muy pequeña. Demasiado pequeño para un eucariota típico y, de hecho, también para muchos microbios demasiado cortos (por ejemplo, los genomas bacterianos son típicamente Mb). Esto indica que el genoma podría ser de un virus.

In [ ]:
print("GC content (%)", GC(record.seq))

GC content (%) 37.97277865097148


In [ ]:
rs2 = Seq("",None)
rs2.seq = record.seq[0:40]

In [ ]:
rs2.seq

Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAAC')

El contenido de GC es 0,38, por lo que la secuencia es algo rica en AT, pero dentro de un rango "normal".

## Comparar con otras secuencias

Usemos BLAST para alinear la secuencia desconocida con otras secuencias anotadas en la base de datos NCBI nt, que contiene secuencias de muchas especies diferentes del árbol de la vida.

Esto puede tomar ~10 minutos ya que estamos haciendo una búsqueda en línea contra muchas secuencias (para consultas más grandes, sería sensato ejecutar BLAST localmente en su lugar; consulte `Bio.Blast.Applications`) El contenido de GC es 0.38, por lo que la secuencia es algo Rico en AT, pero dentro de un rango 'normal'.

In [ ]:
%%time
result_handle = NCBIWWW.qblast("blastn", "nt", rs2.seq)

CPU times: user 309 ms, sys: 41.1 ms, total: 351 ms
Wall time: 1min 1s


Procesamos el resultado con uno de los analizadores genéricos de  Biopython

In [ ]:
blast_qresult = SearchIO.read(result_handle, "blast-xml")

NameError: ignored

In [ ]:
print(blast_qresult)

Program: blastn (2.13.0+)
  Query: No (40)
         definition line
 Target: nt
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  gi|2209248157|gb|ON011633.1|  Severe acute respiratory ...
            1      1  gi|2209244783|gb|ON011417.1|  Severe acute respiratory ...
            2      1  gi|2209244346|gb|ON011391.1|  Severe acute respiratory ...
            3      1  gi|2209243042|gb|ON011314.1|  Severe acute respiratory ...
            4      1  gi|2209241724|gb|ON011239.1|  Severe acute respiratory ...
            5      1  gi|2209241394|gb|ON011221.1|  Severe acute respiratory ...
            6      1  gi|2209240014|gb|ON011148.1|  Severe acute respiratory ...
            7      1  gi|2209150490|gb|ON009426.1|  Severe acute respiratory ...
            8      1  gi|2209150477|gb|ON009425.1|  Severe acute respir

Esas descripciones están truncadas, vamos a verlas en su totalidad, solo para los primeros 5 registros

In [ ]:
[hit.description for hit in blast_qresult[:5]]

['Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/CA-CDPH-2000053397/2021, complete genome',
 'Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/CA-CDPH-2000002421/2020 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope protein (E), membrane glycoprotein (M), ORF6 protein (ORF6), ORF7a protein (ORF7a), ORF7b (ORF7b), ORF8 protein (ORF8), nucleocapsid phosphoprotein (N), and ORF10 protein (ORF10) genes, complete cds',
 'Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/CA-CDPH-2000002390/2020 ORF1ab polyprotein (ORF1ab), ORF1a polyprotein (ORF1ab), surface glycoprotein (S), ORF3a protein (ORF3a), envelope protein (E), membrane glycoprotein (M), ORF6 protein (ORF6), ORF7a protein (ORF7a), ORF7b (ORF7b), ORF8 protein (ORF8), and nucleocapsid phosphoprotein (N) genes, complete cds; and ORF10 gene, complete sequence',
 'Severe acute respiratory s

Bueno, eso parece bastante concluyente, sin hacer ningún análisis cuantitativo, ya es muy probable que tengamos un genoma de coronavirus aquí, ¡específicamente SARS2-CoV-2 que fue la causa de la pandemia de COVID19!

Echemos un vistazo al primer resultado con un poco más de detalle para comprobar algunas de las métricas de alineamiento.

In [ ]:
first_hit = blast_qresult[0]

In [ ]:
first_hit.description

'Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/CA-CDPH-2000053397/2021, complete genome'

In [ ]:
first_hsp = first_hit[0]
print(first_hsp.evalue, first_hsp.bitscore)

3.2902e-10 73.4211


In [ ]:
print(first_hsp.aln)

Alignment with 2 rows and 40 columns
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAAC No
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAAC gi|2209248157|gb|ON011633.1|


El alineamiento parece de alta calidad y no simplemente un golpe de suerte.

Podríamos ver/guardar el alineamiento de secuencia completa, con fines ilustrativos, aquí están solo los primeros 100 caracteres en formato FASTA

In [ ]:
print(first_hsp.aln[:100])

Alignment with 2 rows and 40 columns
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAAC No
ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAAC gi|2209248157|gb|ON011633.1|


## Extraer anotaciones en la secuencia del genoma coincidente

Extraigamos un poco más de metadatos estructurados en la secuencia homóloga de secuencia coincidente superior usando NCBI Entrez a través de Biopython para extraer un archivo GenBank

In [ ]:
NCBI_id = first_hit.id.split('|')[3]
NCBI_id

'ON011633.1'

In [ ]:
Entrez.email = "gustavo.isaza@ucaldas.edu.co"  # Always tell NCBI who you are

In [ ]:
handle = Entrez.efetch(db="nucleotide", id= NCBI_id, retmode="text", rettype="gb")

In [ ]:
genbank_record = SeqIO.read(handle, "genbank")

In [ ]:
genbank_record

SeqRecord(seq=Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...TGC'), id='ON011633.1', name='ON011633', description='Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/CA-CDPH-2000053397/2021, complete genome', dbxrefs=['BioProject:PRJNA750736', 'BioSample:SAMN26727394'])

Hay mucha información en el registro del banco de generación si sabe dónde encontrarla...

In [ ]:
print("Is it single or double stranded and a DNA or RNA virus?\n",
      genbank_record.annotations["molecule_type"])

Is it single or double stranded and a DNA or RNA virus?
 RNA


In [ ]:
print("What is the full NCBI taxonomy of this virus?\n",
      genbank_record.annotations["taxonomy"])

What is the full NCBI taxonomy of this virus?
 ['Viruses', 'Riboviria', 'Orthornavirae', 'Pisuviricota', 'Pisoniviricetes', 'Nidovirales', 'Cornidovirineae', 'Coronaviridae', 'Orthocoronavirinae', 'Betacoronavirus', 'Sarbecovirus']


In [ ]:
print("What are the relevant references/labs who generated the data?\n")
for reference in genbank_record.annotations["references"]:
    print(reference)

What are the relevant references/labs who generated the data?

location: [0:29818]
authors: Smith,E.
title: Direct Submission
journal: Submitted (17-MAR-2022) IDLB VRDL/COVIDNet, California Department of Public Health, 850 Marina Bay Blvd, Richmond, CA 94804, USA
medline id: 
pubmed id: 
comment: 



Ahora podemos leer más sobre el virus y los datos de origen siguiendo estas referencias y las búsquedas de Google adecuadas.

Tenga en cuenta que a partir de esta identificación, también podríamos encontrar el registro aquí (https://www.ncbi.nlm.nih.gov/nuccore/NC_045512.2/) en el sitio web de NCBI.

## Análisis a nivel de proteinas

Podríamos estar interesados en las secuencias de genes/proteínas, no solo en el genoma completo. Es posible recuperar las secuencias de codificación de proteínas (CDS) del registro Genbank

In [ ]:
len(genbank_record.features)

55

In [ ]:
{feature.type for feature in genbank_record.features}

{'CDS', 'gene', 'mat_peptide', 'source', 'stem_loop'}

In [ ]:
CDSs = [feature for feature in genbank_record.features if feature.type == "CDS"]
len(CDSs)

12

Miremos la primera proteína y extraigamos la secuencia subyacente.

In [ ]:
CDSs[0].qualifiers["gene"]

['ORF1ab']

In [ ]:
protein_seq = Seq(CDSs[0].qualifiers["translation"][0])

In [ ]:
protein_seq

Seq('MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGTCGLV...VNN')

In [ ]:
print("Does the sequence begin with a start codon?\n",
      protein_seq.startswith("M"))

Does the sequence begin with a start codon?
 True


In [ ]:
print("Protein sequence length in amino acids", len(protein_seq))

Protein sequence length in amino acids 7096


Es una proteína larga para un virus. Revisemos la anotación

In [ ]:
CDSs[0].qualifiers["product"]

['ORF1ab polyprotein']

Así que parece que es una poliproteína, lo que explica por qué es una proteína relativamente larga. Las poliproteínas son una característica típica de algunos genomas virales donde se unen proteínas más pequeñas, proporcionando una organización particular del proteoma viral.

## Qué sigue?

Los próximos pasos lógicos a nivel del genoma podrían incluir la creación de una alineación de secuencias múltiples a partir de muchos genomas de coronavirus (revise los wrappers/analizadores de Biopython para `Clustal` y `Mafft` y `Bio.Align`/`Bio.parirwise2` más `Bio.AlignIO `), construir una filogenia con una herramienta externa como [iq-tree](http://www.iqtree.org/) y luego ver el árbol con `Bio.Phylo`, el [kit de herramientas ete3](http:// etetoolkit.org/), o [Jalview](https://www.jalview.org/).

Otros análisis del nivel de proteínas podrían implicar la creación de árboles de proteínas, la anotación de las proteínas y la visualización (por ejemplo, "Bio.Graphics"), la realización de análisis de tasa evolutiva (por ejemplo, "Bio.Phylo.PAML"), la observación de la estructura de las proteínas, la agrupación y mucho más.

Este tipo de análisis puede aportar información biológica y epidemiológica útil, muy importante para este coronavirus en situación de brote. Por ejemplo, permitir el seguimiento de cómo se propaga el brote e indicar las medidas adecuadas de control de infecciones, aunque siempre se requiere cautela en la interpretación de los resultados. Consulte [Nextstrain](https://nextstrain.org/ncov) para obtener un excelente recurso de este tipo.

Tenga en cuenta que hay toneladas de otras funciones en Biopython, esta es solo una fracción muy pequeña de los módulos, consulte [Taller de Biopython de Peter Cock] (https://github.com/peterjc/biopython_workshop) y la extensa [documentación oficial del tutorial] (http://biopython.org/DIST/docs/tutorial/Tutorial.html).